In [1]:
"""Federated Learning Model"""
import sys
import os
from collections import OrderedDict
from typing import List

import flwr as fl
import joblib
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import StratifiedShuffleSplit
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

sys.path.append('/home/hadih/repos/FIDS/src/')

from config import FederatedLocation
from utils import Model, straitified_split
from models.Classifier import Classifier

DEVICE = torch.device("cuda")

/home/hadih/.cache/pypoetry/virtualenvs/fids-rgvgQkLu-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-26 01:56:17,271	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
def load_datasets(location=FederatedLocation,):
    """Load datasets"""
    # trainloaders = []
    # valloaders = []
    trainData = []

    # oe = OneHotEncoder(sparse=False)

    data = joblib.load('/home/hadih/repos/FIDS/data/processed/train.pkl')
    chunks = straitified_split(
        data["X"], data["y"], location.clients_number
    )
    
    del data
    for X, y in chunks:
        # X, y = torch.from_numpy(X.astype(np.float32)), torch.tensor(y)
        X = X.astype(np.float32)
        assert X.shape[0] == y.shape[0]
        # ds = TensorDataset(X, y)
        # len_val = len(ds) // 10  # 10 % validation set
        # len_train = len(ds) - len_val
        # lengths = [len_train, len_val]
        # ds_train, ds_val = random_split(
        #     ds, lengths, torch.Generator().manual_seed(42)
        # )
        # trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        trainData.append((X, y))
        # valloaders.append(DataLoader(ds_val, batch_size=32))
        del X, y, # ds, ds_train, ds_val, len_val, len_train, lengths

    del chunks
    testset = joblib.load('/home/hadih/repos/FIDS/data/processed/test.pkl')
    # X, y = torch.from_numpy(testset["X"].astype(np.float32)), torch.tensor(testset["y"])
    # testloader = DataLoader(TensorDataset(X, y), batch_size=32)

    X, y = testset["X"].astype(np.float32), testset["y"]

    del testset
    return trainData, (X, y) # trainloaders, valloaders, testloader


def evaluate(y_true, y_pred):
    # sns.annot_kws={"size": 16}
    # sns.heatmap(pd.DataFrame(confusion_matrix(y_true, y_pred)), annot=True, annot_kws={"size": 9})
    print(classification_report(y_true, y_pred))

In [3]:
# trainloaders, valloaders, testloader = load_datasets()
# trainloader = trainloaders[0]
# valloader = valloaders[0]

trainData, (X_test, y_test) = load_datasets()

In [4]:
for method in ['softmax', 'cnn2', 'cnn5', 'nn3', 'nn5']:
    print(method)
    for i, (X, y) in enumerate(trainData):
        print('Client ', i)
        clf = Classifier(key=f'_{method}_client_{i}', method=method)
        clf.fit(X, y)
        y_pred = clf.predict(X_test)
        evaluate(y_test, y_pred)

softmax
Client  0
Loaded models/Classifier_logs_softmax_client_0 model trained with batch_size = 100, seen 0 epochs and 1349 mini batches


/home/hadih/.cache/pypoetry/virtualenvs/fids-rgvgQkLu-py3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hadih/.cache/pypoetry/virtualenvs/fids-rgvgQkLu-py3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hadih/.cache/pypoetry/virtualenvs/fids-rgvgQkLu-py3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        98
           1       0.92      0.99      0.95      6401
           2       0.82      0.84      0.83       515
           3       0.98      0.90      0.94     11554
           4       0.64      0.73      0.68       275
           5       0.32      0.55      0.40       290
           6       0.69      0.51      0.58       397
           8       0.00      0.00      0.00         2
           9       0.97      0.99      0.98      7947
          10       0.88      0.53      0.66       295
          11       0.22      0.85      0.35        75
          12       0.00      0.00      0.00         1
          13       0.00      0.00      0.00        33

    accuracy                           0.93     27883
   macro avg       0.49      0.53      0.49     27883
weighted avg       0.94      0.93      0.93     27883

Client  1
Loaded models/Classifier_logs_softmax_client_1 model trained with bat

/home/hadih/.cache/pypoetry/virtualenvs/fids-rgvgQkLu-py3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:2394: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
/home/hadih/.cache/pypoetry/virtualenvs/fids-rgvgQkLu-py3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hadih/.cache/pypoetry/virtualenvs/fids-rgvgQkLu-py3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hadih/.cache/pypoetry/virtualen

              precision    recall  f1-score   support

           0       0.04      0.30      0.06        98
           1       0.94      0.97      0.96      6401
           2       0.72      0.92      0.81       515
           3       0.97      0.90      0.93     11554
           4       0.41      0.70      0.52       275
           5       0.31      0.34      0.33       290
           6       0.80      0.63      0.71       397
           8       0.00      0.00      0.00         2
           9       0.99      0.95      0.97      7947
          10       0.85      0.62      0.72       295
          11       0.13      0.27      0.17        75
          12       0.00      0.00      0.00         1
          13       0.00      0.00      0.00        33

    accuracy                           0.91     27883
   macro avg       0.47      0.51      0.47     27883
weighted avg       0.94      0.91      0.92     27883

Client  2
Loaded models/Classifier_logs_softmax_client_2 model trained with bat

...

/home/hadih/.cache/pypoetry/virtualenvs/fids-rgvgQkLu-py3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hadih/.cache/pypoetry/virtualenvs/fids-rgvgQkLu-py3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hadih/.cache/pypoetry/virtualenvs/fids-rgvgQkLu-py3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

              precision    recall  f1-score   support

           0       0.89      0.58      0.70        98
           1       0.97      0.99      0.98      6401
           2       0.98      0.97      0.97       515
           3       0.97      0.99      0.98     11554
           4       0.94      0.65      0.77       275
           5       0.77      0.80      0.79       290
           6       0.99      0.51      0.67       397
           8       0.00      0.00      0.00         2
           9       0.97      0.99      0.98      7947
          10       1.00      0.50      0.67       295
          11       0.66      0.87      0.75        75
          12       0.00      0.00      0.00         1
          13       0.00      0.00      0.00        33

    accuracy                           0.97     27883
   macro avg       0.70      0.60      0.64     27883
weighted avg       0.97      0.97      0.97     27883

Client  2


/home/hadih/.cache/pypoetry/virtualenvs/fids-rgvgQkLu-py3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:2394: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


              precision    recall  f1-score   support

           0       0.95      0.58      0.72        98
           1       1.00      0.98      0.99      6401
           2       0.99      0.98      0.98       515
           3       0.97      1.00      0.98     11554
           4       0.91      0.80      0.85       275
           5       0.90      0.78      0.84       290
           6       1.00      0.51      0.67       397
           8       0.00      0.00      0.00         2
           9       0.97      1.00      0.98      7947
          10       1.00      0.50      0.67       295
          11       0.66      0.87      0.75        75
          12       0.00      0.00      0.00         1
          13       1.00      0.03      0.06        33

    accuracy                           0.97     27883
   macro avg       0.80      0.62      0.65     27883
weighted avg       0.98      0.97      0.97     27883



/home/hadih/.cache/pypoetry/virtualenvs/fids-rgvgQkLu-py3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hadih/.cache/pypoetry/virtualenvs/fids-rgvgQkLu-py3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hadih/.cache/pypoetry/virtualenvs/fids-rgvgQkLu-py3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

In [8]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from tqdm import tqdm


class Net(nn.Module):
    """Model Class"""

    def __init__(self, input_dim=76, num_classes=14) -> None:
        super(Net, self).__init__()
        self.input_dim = input_dim
        self.num_classes = num_classes
        
        layers = []
        layers.append(nn.Linear(input_dim,128))

        layers.append(nn.BatchNorm1d(128))
        layers.append(nn.ReLU(True))
        layers.append(nn.Linear(128,256))
        
        layers.append(nn.BatchNorm1d(256))
        layers.append(nn.Dropout(p=0.3))
        layers.append(nn.ReLU(True))
        layers.append(nn.Linear(256,256))
        
        layers.append(nn.BatchNorm1d(256))
        layers.append(nn.Dropout(p=0.4))
        layers.append(nn.ReLU(True))
        layers.append(nn.Linear(256,128))

        layers.append(nn.BatchNorm1d(128))
        layers.append(nn.Dropout(p=0.5))
        layers.append(nn.ReLU(True))        
        layers.append(nn.Linear(128,num_classes))
        layers.append(nn.LogSoftmax())

        self.model = nn.Sequential(*layers).to(DEVICE)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for features, labels in tqdm(trainloader, desc=f'Epock {epoch+1}: '):
            features, labels = features.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(features)
            _, predicted = torch.max(outputs.data, 1)
            loss = criterion(
                net(features),
                labels
            )
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(
            f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}"
        )


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss, predictions, true = 0, 0, 0.0, [], []
    net.eval()
    with torch.no_grad():
        for data_points, labels in tqdm(testloader):
            true.append(labels)
            data_points, labels = data_points.to(DEVICE), labels.to(DEVICE)
            outputs = net(data_points)
            _, predicted = torch.max(outputs.data, 1)
            predictions.append(predicted.cpu().tolist())
            loss += criterion(outputs, labels).item()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy, true, predictions

In [10]:
def get_parameters(net) -> List[np.ndarray]:
    """Get params"""
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    """Set params"""
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


class FlowerClient(fl.client.NumPyClient):
    """Fedrated Client"""

    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        """Get params"""
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        """fit client model"""
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        """evalute"""
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


class FedratedModel(Model):
    """Federated Architecture Model"""

    def __init__(self, trainloaders, valloaders, testloader) -> None:
        super().__init__()
        self.trainloaders, self.valloaders, self.testloader = trainloaders, valloaders, testloader

    def client_fn(self, cid) -> FlowerClient:
        net = Net().to(DEVICE)
        trainloader = self.trainloaders[int(cid)]
        valloader = self.valloaders[int(cid)]
        return FlowerClient(cid, net, trainloader, valloader)

    def train(self):
        """train fedrated global model"""
        # Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
        # client_resources = None
        # if DEVICE.type == "cuda":
        #     client_resources = {"num_gpus": 1}

        fl.simulation.start_simulation(
            client_fn=self.client_fn,
            num_clients=3,
            config=fl.server.ServerConfig(num_rounds=3),
            # client_resources=client_resources,
        )

In [11]:
f = FedratedModel(trainloaders=trainloaders, valloaders=valloaders, testloader=testloader)
f.train()

INFO flwr 2023-08-23 13:58:30,152 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-08-23 13:58:32,497	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-08-23 13:58:34,093 | app.py:180 | Flower VCE: Ray initialized with resources: {'node:172.25.119.181': 1.0, 'CPU': 16.0, 'object_store_memory': 1657331712.0, 'memory': 3314663424.0, 'node:__internal_head__': 1.0}
INFO flwr 2023-08-23 13:58:34,093 | server.py:86 | Initializing global parameters
INFO flwr 2023-08-23 13:58:34,094 | server.py:273 | Requesting initial parameters from one random client
ERROR flwr 2023-08-23 13:58:35,592 | ray_client_proxy.py:72 | ray::launch_and_get_parameters() (pid=43626, ip=172.25.119.181)
  At least one of the input arguments for this task could not be computed:
ray.exceptions.RaySystemError: System error: No module named 'utils'
traceback: Traceback (most recent call last):
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

RayTaskError: [36mray::launch_and_get_parameters()[39m (pid=43626, ip=172.25.119.181)
  At least one of the input arguments for this task could not be computed:
ray.exceptions.RaySystemError: System error: No module named 'utils'
traceback: Traceback (most recent call last):
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
          ^^^^^^^^^^^^^^^^^^^^^
ModuleNotFoundError: No module named 'utils'